In [1]:
import geopandas
import pandas as pd
import numpy as np
from shapely.ops import unary_union
from pyproj import Geod
from polygons import split_polygon, snake_sort, ns
from functools import cmp_to_key

In [2]:
gdf = geopandas.read_file("../data/lots2.geojson")

In [3]:
df = pd.read_csv('../data/cook_county_cleaned3.csv')

In [4]:
df.head()

,Unnamed: 0,Cleaned Name,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,...,Total Price,Type of Sale,Date of Purchase,Volume,Page,Shapefile Object ID,Voided,Lot Matches,Name Matches,Duplicate
0,0,ABBE CHRISTIAN,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,...,160.08,SC,09/01/1852,819,64,8570.0,False,NaN,0,False
1,1,ABBOTT ALLISON,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,...,99.65,FD,02/17/1841,687,191,4828.0,False,NaN,1,False
2,2,ABBOTT JOSEPH,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,...,360,CN,05/09/1853,L7A,180,7367.0,False,2,2,False
3,3,ABBOTT SAMUEL S,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,...,200,FD,11/27/1838,687,194,5097.0,False,NaN,3;4,False
4,4,ABBOTT SAMUEL S,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,...,200,FD,11/17/1838,687,194,5100.0,False,NaN,3;4,False


In [5]:
gdf.head()

,TWPNUM,RNGNUM,SECTION,PART,INDIAN_BOUNDARY,QUARTER_OBJECTID,fracl,TOWNSHIP,RANGE,SECTN,QUARTERSEC,IBL,SHAPE_Length,SHAPE_Area,QUARTER,geometry
0,35,13,1,NENE,0,3137.0,0,35,13,1,N.E.1/4,0,10625.039725,7.049872e+06,NE,"POLYGON ((-87.67485 41.55721, -87.67484 41.555..."
1,35,13,1,NENW,0,3137.0,0,35,13,1,N.E.1/4,0,10625.039725,7.049872e+06,NE,"POLYGON ((-87.68458 41.55720, -87.67971 41.557..."
2,35,13,1,NESE,0,3137.0,0,35,13,1,N.E.1/4,0,10625.039725,7.049872e+06,NE,"POLYGON ((-87.67481 41.55354, -87.67481 41.553..."
3,35,13,1,NESW,0,3137.0,0,35,13,1,N.E.1/4,0,10625.039725,7.049872e+06,NE,"POLYGON ((-87.68444 41.55024, -87.68444 41.550..."
4,35,13,1,NWNE,0,3138.0,0,35,13,1,N.W.1/4,0,10608.073059,7.036351e+06,NW,"POLYGON ((-87.68458 41.55720, -87.68449 41.553..."


In [6]:
for col in ['TWPNUM', 'RNGNUM', "SECTION"]:
    gdf[col] = gdf[col].astype(int)
        
df['Acres'] = df['Acres'].str.replace(" ", "")
df['Acres'] = df['Acres'].astype(float)

In [7]:
df.loc[df['Aliquot Parts or Lot'].str.contains('NIBL'), "IBL"] = 'NIBL'
df.loc[df['Aliquot Parts or Lot'].str.contains('SIBL'), "IBL"] = 'SIBL'
df.loc[df['IBL'].isnull(), "IBL"] = ''
gdf.loc[gdf['INDIAN_BOUNDARY'] == '0', "INDIAN_BOUNDARY"] = ''

In [8]:
lots = ["LOT" + str(num) for num in range(32, 0, -1)]
lots.extend(['LOT0' + str(num) for num in range(1,10)])
print(lots)

['LOT32', 'LOT31', 'LOT30', 'LOT29', 'LOT28', 'LOT27', 'LOT26', 'LOT25', 'LOT24', 'LOT23', 'LOT22', 'LOT21', 'LOT20', 'LOT19', 'LOT18', 'LOT17', 'LOT16', 'LOT15', 'LOT14', 'LOT13', 'LOT12', 'LOT11', 'LOT10', 'LOT9', 'LOT8', 'LOT7', 'LOT6', 'LOT5', 'LOT4', 'LOT3', 'LOT2', 'LOT1', 'LOT01', 'LOT02', 'LOT03', 'LOT04', 'LOT05', 'LOT06', 'LOT07', 'LOT08', 'LOT09']


In [9]:
test = "TESTVOIDVOTB"
to_strip = ['2', '4', 'VOID', 'VO', "V", 'PRA', 'FR', 'TB', "MA", "TE", 'NIBL', 'SIBL','SC','SK',\
            "ROAD", "SR" "CALR","R", 'SCANAL','NCANAL','CANAL', 'SECT','SEC', 'DEA', 'FEED', 'PT']

for string in to_strip:
    test = test.replace(string, "")

In [10]:
def strip_common_strings(lot):
    for string in lots:
        lot = lot.replace(string, "")
    for string in to_strip:
        lot = lot.replace(string, "")
    
    return lot

In [11]:
new_lots = df['Aliquot Parts or Lot'].apply(strip_common_strings)

In [12]:
df['part'] = new_lots

In [13]:
class TooManyMatches(Exception):
    pass

class NoMatch(Exception):
    pass

In [14]:
def get_part_geometry(row, lot):
    twp = int(row['Township'][:-1])
    rng = int(row["Range"][:-1])
    sect = int(row['Section Number'])

    geometries = gdf.loc[(gdf['TWPNUM'] == twp)\
      &(gdf['RNGNUM'] == rng)\
      &(gdf['SECTION'] == sect)\
      &(gdf["PART"] == lot)
      &(gdf['INDIAN_BOUNDARY'] == row['IBL']),\
      'geometry'
     ].values

    if len(geometries) > 1:
        print(row)
        print(lot)
        raise TooManyMatches
        
    if len(geometries) < 1:
        raise NoMatch
    
    return geometries[0]
    

def get_parts(row):
    legit_sections = ['NW', "NE", "SW", "SE"]
    legit_directions = {'N' : ['NE', "NW"], "E": ['NE', "SE"], "S": ['SE', 'SW'], "W": ['NW', 'SW']}
    part = row['part']

    try:
        if len(part) > 1 and len(part) < 5:
            first_sect = part[-2:] 

            if first_sect not in legit_sections:
                return None

            if len(part) == 4:
                last_sect = part[:-2]

                if last_sect not in legit_sections:
                    return None

                return get_part_geometry(row, first_sect + last_sect)


            if len(part) == 3:
                direc = part[0]
                last_sects = legit_directions.get(direc)

                if last_sects:  
                    geometries = [get_part_geometry(row, first_sect + last_sect) for last_sect in last_sects]                    
                    return unary_union(geometries)

            if len(part) == 2:
                geometries = [get_part_geometry(row, first_sect + last_sect) for last_sect in legit_sections]
                return unary_union(geometries)
            
        elif len(part) == 1 and legit_directions.get(part):
            first_sects = legit_directions.get(part)
            geometries = []
            for first_sect in first_sects:
                for last_sect in legit_sections:
                    geometries.append(get_part_geometry(row, first_sect+last_sect))
                        
            return unary_union(geometries)
        
        elif row['Acres'] > 450:
            geometries = []
            for first_sect in legit_sections:
                for last_sect in legit_sections:
                    geometries.append(get_part_geometry(row, first_sect+last_sect))
            
            return unary_union(geometries)
                
    except NoMatch:
        return None        
    
    
    return None

geod = Geod(ellps="WGS84")

def get_geometry(row):
    geometry = get_parts(row)
    
    if geometry:
        return geometry
    
    row['IBL'] = 'NIBL'
    geometry1 = get_parts(row)
    row['IBL'] = 'SIBL'
    geometry2 = get_parts(row)

    if not geometry1 and geometry2:
        return geometry2
    elif not geometry2 and geometry1:
        return geometry1
    elif geometry1 and geometry2:
        area1 = abs(geod.geometry_area_perimeter(geometry1)[0]) * 0.000247105
        area2 = abs(geod.geometry_area_perimeter(geometry2)[0]) * 0.000247105
        diff1 = abs(row['Acres'] - area1)
        diff2 = abs(row['Acres'] - area2)
        
        if diff1 < diff2:
            return geometry1
        elif diff2 < diff1:
            return geometry2
    
    return None
    

In [15]:
geometries = df.apply(get_geometry, axis=1)

/Users/davidhackett/opt/anaconda3/envs/public_land_records/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1983: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [16]:
df['geometry'] = geometries

In [17]:
df.loc[(df['Township'] == '37N')&(df['Range'] == '14E')&(df['Section Number'] == 16)]

,Unnamed: 0,Cleaned Name,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,...,Volume,Page,Shapefile Object ID,Voided,Lot Matches,Name Matches,Duplicate,IBL,part,geometry
1432,1448,BUCKNELL THOMAS,BUCKNELL THOMAS,UNKNOWN,NaN,LOT4,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,1448;1449;1450;1451;1452,False,,,None
1433,1449,BUCKNELL THOMAS,BUCKNELL THOMAS,UNKNOWN,NaN,LOT12,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,1448;1449;1450;1451;1452,False,,,None
1434,1450,BUCKNELL THOMAS,BUCKNELL THOMAS,UNKNOWN,NaN,LOT11,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,1448;1449;1450;1451;1452,False,,,None
1435,1451,BUCKNELL THOMAS,BUCKNELL THOMAS,UNKNOWN,NaN,LOT5,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,1448;1449;1450;1451;1452,False,,,None
1436,1452,BUCKNELL THOMAS,BUCKNELL THOMAS,UNKNOWN,NaN,LOT6,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,1448;1449;1450;1451;1452,False,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11145,12093,TULLY MURRAY F,TULLY MURRAY F,UNKNOWN,NaN,LOT32,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,12091;12092;12093,False,,,None
11229,12177,VANDERSIFDEW SENDER,VANDERSIFDEW SENDER,UNKNOWN,NaN,LOT59,16.0,37N,14E,3.0,...,819,97,7645.0,False,NaN,12177,False,,9,None
11615,12589,WELP HENRY,WELP HENRY,UNKNOWN,NaN,LOT22,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,12589,False,,,None
11724,12699,WHEELER ANDREW B,WHEELER ANDREW B,UNKNOWN,NaN,LOT13,16.0,37N,14E,3.0,...,819,65,7645.0,False,NaN,12699;12700;12701;12702,False,,,None


In [18]:
geo371416 = unary_union(gdf.loc[(gdf['TWPNUM'] == 37)&(gdf['RNGNUM'] == 14)&(gdf['SECTION'] == 16), 'geometry'])

geos371416 = list(split_polygon(geo371416, 4, 16).geoms)
geos371416 = snake_sort(geos371416, 4, "NE")

In [19]:
for idx, row in df.loc[(df['Township'] == '37N')&(df['Range'] == '14E')&(df['Section Number'] == 16)].iterrows():
    lot = row['Aliquot Parts or Lot']
    num = int(lot.replace("LOT", ""))
    df.loc[idx, 'geometry'] = geos371416[num-1]

In [20]:
geo391316 = unary_union(gdf.loc[(gdf['TWPNUM'] == 37)&(gdf['RNGNUM'] == 14)\
                                &(gdf['SECTION'] == 16)&(gdf['PART'].str.contains('N')), 'geometry'])

geos391316 = list(split_polygon(geo391316, 10, 25).geoms)
geos391316 = snake_sort(geos391316, 25, "NE", True)

In [21]:
len(geos391316)

251

In [22]:
for idx, row in df.loc[(df['Township'] == '39N')&(df['Range'] == '13E')\
                       &(df['Section Number'] == 16)&(df['Aliquot Parts or Lot'].str.contains("NPT"))].iterrows():
    lot = row['Aliquot Parts or Lot']
    num = int(lot.replace("LOT", "").replace("LL", "").replace("L", "").replace("NPT", ""))
    df.loc[idx, 'geometry'] = geos391316[num-1]

In [23]:
geo391433 = unary_union(gdf.loc[(gdf['TWPNUM'] == 39)&(gdf['RNGNUM'] == 14)\
                                &(gdf['SECTION'] == 33), 'geometry'])

geos391433 = list(split_polygon(geo391433, 8, 4).geoms)
geos391433 = snake_sort(geos391433, 8, "NE")

In [24]:
for idx, row in df.loc[(df['Township'] == '39N')&(df['Range'] == '14E')&(df['Section Number'] == 33)].iterrows():
    lot = row['Aliquot Parts or Lot']
    for string in ["LOT", "BL", "NE", "SE", "SW", "NW", "S", "N"]:
        lot = lot.replace(string, "")
    num = int(lot)
    df.loc[idx, 'geometry'] = geos391433[num-1]

In [25]:
geos401416nw = unary_union(gdf.loc[(gdf['TWPNUM'] == 40)&(gdf['RNGNUM'] == 14)&(gdf['SECTION'] == 16)&(gdf['PART'].isin(['NWNW', 'NWSW'])), 'geometry'])
geos401416nw = list(split_polygon(geos401416nw, 1, 4).geoms)

geos401416sw = unary_union(gdf.loc[(gdf['TWPNUM'] == 40)&(gdf['RNGNUM'] == 14)&(gdf['SECTION'] == 16)&(gdf['PART'].isin(['SWNW', 'SWSW'])), 'geometry'])
geos401416sw = list(split_polygon(geos401416sw, 2, 10).geoms)

geos401416 = sorted(geos401416nw, key=cmp_to_key(ns))
geos401416sw = snake_sort(geos401416sw, 10, 'NE', True)

geos401416.extend(geos401416sw)

In [26]:
for idx, row in df.loc[(df['Township'] == '40N')&(df['Range'] == '14E')&(df['Section Number'] == 16)].iterrows():
    lot = row['Aliquot Parts or Lot']
    lot = lot.replace("LOT", "")
    num = int(lot)
    df.loc[idx, 'geometry'] = geos401416[num-1]

In [27]:
geos42916ne = unary_union(gdf.loc[(gdf['TWPNUM'] == 42)&(gdf['RNGNUM'] == 9)&(gdf['SECTION'] == 16)&(gdf['QUARTER'] == 'NE'), 'geometry'])
geos42916ne = list(split_polygon(geos42916ne, 4, 4).geoms)
geos42916ne = snake_sort(geos42916ne, 4, "NE")

In [28]:
for idx, row in df.loc[(df['Township'] == '42N')&(df['Range'] == '09E')\
                       &(df['Section Number'] == 16)&df['geometry'].isnull()].iterrows():
    lot = row['Aliquot Parts or Lot']
    lot = lot.replace("LOT", "").replace("E2N2NENE", '')
    num = int(lot)
    df.loc[idx, 'geometry'] = geos42916ne[num-1]

In [29]:
geos391431e = unary_union(gdf.loc[(gdf['TWPNUM'] == 39)&(gdf['RNGNUM'] == 14)\
                                  &(gdf['SECTION'] == 31)&(gdf['QUARTER'].str.contains('E')), 'geometry'])
geos391431e = list(split_polygon(geos391431e, 4, 8).geoms)
geos391431e = snake_sort(geos391431e, 4, "NE")

In [30]:
for idx, row in df.loc[(df['Township'] == '39N')&(df['Range'] == '14E')\
                       &(df['Section Number'] == 31)&df['Aliquot Parts or Lot'].str.contains('E2')].iterrows():
    lot = row['Aliquot Parts or Lot']
    lot = lot.replace("LOT", "").replace("E2", '')
    num = int(lot)
    if num < 7:
        df.loc[idx, 'geometry'] = geos391431e[num-1]
    elif num < 9:
        df.loc[idx, 'geometry'] = geos391431e[num-2]
    elif num < 11:
        df.loc[idx, 'geometry'] = geos391431e[num-3]
    elif num < 16:
        df.loc[idx, 'geometry'] = geos391431e[num-4]
    else:
        df.loc[idx, 'geometry'] = geos391431e[num-5]

In [31]:
geos381416e = unary_union(gdf.loc[(gdf['TWPNUM'] == 38)&(gdf['RNGNUM'] == 14)\
                                  &(gdf['SECTION'] == 16)&(gdf['QUARTER'].str.contains('E')), 'geometry'])
geos381416e = list(split_polygon(geos381416e, 2, 14).geoms)
geos381416e = snake_sort(geos381416e, 14, "NE", True)

geos381416we = unary_union(gdf.loc[(gdf['TWPNUM'] == 38)&(gdf['RNGNUM'] == 14)\
                                  &(gdf['SECTION'] == 16)&(gdf['PART'].isin(['NWNE', 'NWSE', 'SWNE', 'SWSE'])), 'geometry'])

geos381416we = list(split_polygon(geos381416we, 1, 8).geoms)
geos381416we = sorted(geos381416we,  key=cmp_to_key(ns))
geos381416 = geos381416e + geos381416we

for q in ['SWSW', 'SWNW', 'NWSW', 'NWNW']:
    geo = gdf.loc[(gdf['TWPNUM'] == 38)&(gdf['RNGNUM'] == 14)\
                                  &(gdf['SECTION'] == 16)&(gdf['PART'] == q), 'geometry'].tolist()[0]
    geos381416.append(geo)
    
len(geos381416)

40

In [32]:
for idx, row in df.loc[(df['Township'] == '38N')&(df['Range'] == '14E')\
                       &(df['Section Number'] == 16)&df['geometry'].isnull()].iterrows():
    
    lot = row['Aliquot Parts or Lot']
    
    for string in ['L', "TRSUBD", "TRSBUD", "TRUSBD"]:
        lot = lot.replace(string, "")
    num = int(lot)
    df.loc[idx, 'geometry'] = geos381416[num-1]

In [33]:
len(df[df['geometry'].isnull()])

3745

In [34]:
df.to_csv("../data/cook_county_with_geometries2.csv")

In [35]:
newgdf = geopandas.GeoDataFrame(df)
#newgdf.set_geometry(col='geometry', inplace=True)
newgdf.to_file("../data/cook_county_with_geometries2.geojson", driver="GeoJSON")

In [36]:
gdf.loc[(gdf['TWPNUM'] == 37)\
      &(gdf['RNGNUM'] == 14)\
      &(gdf['SECTION'] == 19)\
      &(gdf['INDIAN_BOUNDARY'] ==  '')
     ]

,TWPNUM,RNGNUM,SECTION,PART,INDIAN_BOUNDARY,QUARTER_OBJECTID,fracl,TOWNSHIP,RANGE,SECTN,QUARTERSEC,IBL,SHAPE_Length,SHAPE_Area,QUARTER,geometry
4995,37,14,19,NENE,,3925.0,0,37,14,19,N.E.1/4,0,10606.429347,7.030952e+06,NE,"POLYGON ((-87.66656 41.69204, -87.66641 41.692..."
4996,37,14,19,NENW,,3925.0,0,37,14,19,N.E.1/4,0,10606.429347,7.030952e+06,NE,"POLYGON ((-87.67143 41.68899, -87.67143 41.689..."
4997,37,14,19,NESE,,3925.0,0,37,14,19,N.E.1/4,0,10606.429347,7.030952e+06,NE,"POLYGON ((-87.66173 41.68839, -87.66162 41.684..."
4998,37,14,19,NESW,,3925.0,0,37,14,19,N.E.1/4,0,10606.429347,7.030952e+06,NE,"POLYGON ((-87.67131 41.68491, -87.67138 41.687..."
4999,37,14,19,NWNE,,2211.0,0,37,14,19,N.W.1/4,0,10576.372940,7.003337e+06,NW,"POLYGON ((-87.67403 41.69190, -87.67196 41.691..."
5000,37,14,19,NWNW,,2211.0,0,37,14,19,N.W.1/4,0,10576.372940,7.003337e+06,NW,"POLYGON ((-87.68109 41.68927, -87.68109 41.689..."
5001,37,14,19,NWSE,,2211.0,0,37,14,19,N.W.1/4,0,10576.372940,7.003337e+06,NW,"POLYGON ((-87.67141 41.68825, -87.67138 41.687..."
5002,37,14,19,NWSW,,2211.0,0,37,14,19,N.W.1/4,0,10576.372940,7.003337e+06,NW,"POLYGON ((-87.68097 41.68565, -87.68099 41.686..."
5003,37,14,19,SENE,,3927.0,0,37,14,19,S.E.1/4,0,10600.648299,7.023340e+06,SE,"POLYGON ((-87.66406 41.68479, -87.66368 41.684..."
5004,37,14,19,SENW,,3927.0,0,37,14,19,S.E.1/4,0,10600.648299,7.023340e+06,SE,"POLYGON ((-87.67120 41.68117, -87.67121 41.681..."


In [37]:
len(gdf)

15527

In [38]:
geod = Geod(ellps="WGS84")
abs(geod.geometry_area_perimeter(df.loc[2, 'geometry'])[0]) * 0.000247105

40.29407932175147

In [39]:
df.head()

,Unnamed: 0,Cleaned Name,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,...,Volume,Page,Shapefile Object ID,Voided,Lot Matches,Name Matches,Duplicate,IBL,part,geometry
0,0,ABBE CHRISTIAN,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,...,819,64,8570.0,False,NaN,0,False,,NWNE,"POLYGON ((-87.74681 41.61773, -87.74191 41.617..."
1,1,ABBOTT ALLISON,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,...,687,191,4828.0,False,NaN,1,False,,SW,"POLYGON ((-87.79759 41.98283, -87.79762 41.982..."
2,2,ABBOTT JOSEPH,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,...,L7A,180,7367.0,False,2,2,False,,NWSW,"POLYGON ((-87.85677 41.72505, -87.85220 41.725..."
3,3,ABBOTT SAMUEL S,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,...,687,194,5097.0,False,NaN,3;4,False,,SW,"POLYGON ((-87.72767 41.95549, -87.72769 41.955..."
4,4,ABBOTT SAMUEL S,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,...,687,194,5100.0,False,NaN,3;4,False,,SE,"POLYGON ((-87.73746 41.95391, -87.73747 41.954..."
